# The Julia language and C++

### The perfect marriage?

<br/>
### C++Now 2018
<br/>
Keno Fisher, Bart Janssens

# Short bio: Bart
Work: [Royal Military Academy](http://www.rma.ac.be) of Belgium

C++:
* [K-3D](http://www.k-3d.org)
 - [BoostCon 2007 presentation on Boost.Python](http://www.k-3d.org/k3d_wiki/images/8/8b/Extreme_Object_Models_Using_Boost.Python.pdf) by previous maintanter Tim Shead
* [Coolfluid3](https://coolfluid.github.io)
 - [C++Now 2013 presentation on using Boost.Proto with Eigen](https://github.com/boostcon/cppnow_presentations_2013/blob/master/fri/proto-eigen-fem.pdf?raw=true)
 
C++ and Julia:
* CxxWrap.jl
* QML.jl
* Trilinos.jl

# Outline
* Why Julia?
* Julia intro: a C++ programmer's perspective
* CxxWrap.jl
* Cxx.jl
* Code repository: https://github.com/barche/cppnow2018-julia
 


# About Julia
## Why another language?
* Solve the "two language problem":
  - Prototype in a simple language
  - Write production code in a fast language

## So what is it?
* High-level programming language for scientific computing
* Dynamic language
* Strongly typed, with user-defined types and generics
* JIT-compiled using LLVM
* Native interface to C
* Central concept: **multiple dispatch**

## A simple function

In [1]:
function add(a,b)
    return a + b
end

add (generic function with 1 method)

Shorter version:

In [2]:
add(a,b) = a+b

add (generic function with 1 method)

In [3]:
add(1,2)

3

In [4]:
add(1.0, 2.0)

3.0

## Where are the types?

In [5]:
typeof(1)

Int64

In [6]:
typeof(add(1,2))

Int64

In [7]:
typeof(add(1.0,2))

Float64

## A look at the assembly
Each combination of types for the arguments compiles a different version of the code:

In [8]:
@code_native add(1,2)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[2]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	leaq	(%rdi,%rsi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


In [9]:
@code_native add(1.0,2.0)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[2]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	addsd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopw	(%rax,%rax)


## Equivalence with C++
The Julia function `add(a,b) = a+b` is equivalent to the following C++ function:

```c++
template<typename A, typename B>
auto add(A a, B b) -> decltype(a + b)
{
    return a + b;
}
```

* Template function valid for any type `A` and `B`
* C++ compiles a new version for every combination of types
* Automatic (static) computation of the return type (`decltype` annotation)

## Types

In [10]:
# Define a type
struct MyNumber
    n::Int
end

In [11]:
# Create an instance
const mynum = MyNumber(2)

MyNumber(2)

## What about our `add` function?

In [12]:
add(mynum, 2)

LoadError: [91mMethodError: no method matching +(::MyNumber, ::Int64)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  +([91m::Complex{Bool}[39m, ::Real) at complex.jl:247
  +([91m::Char[39m, ::Integer) at char.jl:40
  ...[39m

In [13]:
add(a::MyNumber, b) = MyNumber(a.n + b)

add (generic function with 2 methods)

In [14]:
methods(add)

# 2 methods for generic function "add":
add(a::MyNumber, b) in Main at In[13]:1
add(a, b) in Main at In[2]:1

In [15]:
add(mynum, 2)

MyNumber(4)

## Inheritance
Julia supports single inheritance from abstract base types:

In [31]:
abstract type MyBase end
struct MyConcrete <: MyBase
    a::Int
end
# This kind of introspection is built-in:
supertype(MyConcrete)

MyBase

In [17]:
geta(x::MyBase) = x.a
some_a = MyConcrete(3)
geta(some_a)

3

## Constness

In [18]:
some_a.a = 2

LoadError: [91mtype MyConcrete is immutable[39m

In [21]:
# Make a mutable subtype:
mutable struct MutableConcrete <: MyBase
    a::Int
end

mutable_a = MutableConcrete(4)
# Still works:
@show geta(mutable_a)
# We can change it now:
mutable_a.a = 42
geta(mutable_a)

geta(mutable_a) = 4


42

## Generic types

In [29]:
struct Point{T,N}
    coords::NTuple{N,T}
end
p1 = Point((1,2,3))

Point{Int64,3}((1, 2, 3))

In [30]:
p2 = Point((2.0, 3.0))

Point{Float64,2}((2.0, 3.0))

In [28]:
# While this is allowed in C++, it's not in Julia:
struct Combined{T,N1,N2}
    coords::NTuple{N1+N2,T}
end

LoadError: [91mMethodError: no method matching +(::TypeVar, ::TypeVar)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424[39m

## Multiple dispatch
* In Julia, not writing types means any type can be used
* It still tracks the types and computes the correct return type
* Each combination of argument types yields a newly compiled version of the function
* Type computation can happen dynamically (at runtime) or statically (during JIT compilation)
* The static type computation is equivalent to the C++ version
* Deciding which function to call is **multiple dispatch** (static or dynamic)

## Dynamic dispatch

Dynamic dispatch is similar to C++ virtual functions, except that in Julia it dispatches on more than just the first argument. Consider an heterogeneous array:


In [10]:
A1 = [1, 2.0, [3,4]]


3-element Array{Any,1}:
  5          
 0+6im       
   [7.0, 2.0]

In [14]:
# Take the norm of every element
norm.(A1)

3-element Array{Real,1}:
 1  
 2.0
 5.0

## Dynamic dispatch vs. virtual functions
* Because every element has a different type, the `norm` function is chosen at *runtime*, depending on the actual content of the array
* In C++, this can realized using a common base class and a virtual `norm` function.

Let's add a second array:

In [17]:
A2 = [[5, 6.0], 7im, 8]

3-element Array{Any,1}:
   [5.0, 6.0]
 0+7im       
  8          

Now adding this to the previous array:

In [18]:
A1 .+ A2

3-element Array{Any,1}:
     [6.0, 7.0]
 2.0+7.0im     
     [11, 12]  

This performs a dynamic dispatch based on *both* arguments to `+`

## Julia for C++ programmers: summary

|   |Julia|C++|
|---|---|---|
|Types|☑️|☑️|
|Inheritance|Single, abstract|Multiple, concrete|
|Constness|Type level|Variable level|
|Methods part of class |❌|☑️|
|Generic types|☑️|☑️|
|Compile-time type computation|❌|☑️|
|Introspection|☑️|❌|
|Multiple dispatch|static + dynamic|static|

# CxxWrap
* Github page: https://github.com/JuliaInterop/CxxWrap.jl
* Makes use of the native C calling ability of Julia
* Most code for wrapping C++ is written in C++, compiled with your favourite compiler
* Inspired by Boost.Python and pybind11

## Calling C functions from Julia
Using `ccall`:

In [27]:
ccall(:fabs, Float64, (Float64,),-1)

1.0

The overhead is low:

In [28]:
using BenchmarkTools

In [29]:
@btime ccall(:fabs, Float64, (Float64,),-1.0)

  3.612 ns (0 allocations: 0 bytes)


1.0

In [30]:
@btime abs(-1.0)

  2.539 ns (0 allocations: 0 bytes)


1.0

## CxxWrap approach for functions
* First argument to `ccall` can be a function pointer
* For C-like functions: pass pointer directly
* Otherwise: pass a function pointer that has the actual arguments _and_ a pointer to an `std::function` closure

## Example
Code from [`tutorials/cxxwrap/hello_world`](tutorials/cxxwrap/hello_world):
```c++
#include <iostream>
#include <jlcxx/jlcxx.hpp>

void hello() { std::cout << "hello world!" << std::endl; }

JULIA_CPP_MODULE_BEGIN(registry)
  jlcxx::Module& mod = registry.create_module("Hello");

  mod.method("hello", hello);

JULIA_CPP_MODULE_END
```

### CMake:
```cmake
project(Hello)
cmake_minimum_required(VERSION 2.8.12)

find_package(JlCxx REQUIRED)

set(CMAKE_INSTALL_RPATH "${JlCxx_DIR}/../")
set(CMAKE_INSTALL_RPATH_USE_LINK_PATH TRUE)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -std=c++11")

add_library(hello SHARED hello.cpp)
target_link_libraries(hello JlCxx::cxxwrap_julia)
```

## Running in Julia

In [36]:
# Customize this to your actual build directory
buildroot = joinpath(ENV["HOME"], "src/build/cppnow2018");

In [38]:
using CxxWrap

In [43]:
wrap_modules(joinpath(buildroot, "hello/libhello"))

In [44]:
Hello.hello()

hello world!


## Behind the scenes

#### Some timings
Loop over 50 M elements doing `double` multiplication:
```
Julia:
  0.066912 seconds (4 allocations: 160 bytes)
ccall in Julia loop:
  0.104013 seconds (4 allocations: 160 bytes)
CxxWrap.jl call in Julia loop:
  0.106241 seconds (4 allocations: 160 bytes)
C++, loop in the C++ code:
  0.064130 seconds (4 allocations: 160 bytes)
Julia cfunction callback in C++ loop
  0.277433 seconds (4 allocations: 160 bytes)
```

## [Trilinos.jl](https://github.com/barche/Trilinos.jl)

* Interface to some parts of the [Trilinos](https://trilinos.org/) C++ library
* Focus on the Tpetra matrix library and iterative solvers
* Uses CxxWrap.jl
* Example: 2D Laplace in C++ and Julia

### Laplace example:
Solve the equation $\nabla^2 \varphi + f = 0 $ with $f = 2h^2((1-x^2)+(1-y^2))$ and 0 on the boundary:

<img src="images/laplace2d.png" width=700px>

### C++ code
```c++
template<typename MatrixT>
void fill_laplace2d(MatrixT& A, const CartesianGrid& g)
{
  Teuchos::TimeMonitor local_timer(*fill_time);
  const auto& rowmap = *A.getRowMap();
  const local_t n_my_elms = rowmap.getNodeNumElements();

  // storage for the per-row values
  global_t row_indices[5] = {0,0,0,0,0};
  scalar_t row_values[5] = {4.0,-1.0,-1.0,-1.0,-1.0};

  for(local_t i = 0; i != n_my_elms; ++i)
  {
    const global_t global_row = rowmap.getGlobalElement(i);
    const local_t row_n_elems = laplace2d_indices(row_indices, global_row, g);
    row_values[0] = 4.0 - (5-row_n_elems);
    A.replaceGlobalValues(global_row, Teuchos::ArrayView<global_t>(row_indices,row_n_elems), Teuchos::ArrayView<scalar_t>(row_values,row_n_elems));
  }
}
```

### Julia code
```julia
function fill_laplace2d!(A, g::CartesianGrid)
  rowmap = Tpetra.getRowMap(A)
  n_my_elms = Tpetra.getNodeNumElements(rowmap)

  # storage for the per-row values
  row_indices = [0,0,0,0,0]
  row_values = [4.0,-1.0,-1.0,-1.0,-1.0]

  for i in 0:n_my_elms-1
    global_row = Tpetra.getGlobalElement(rowmap,i)
    row_n_elems = laplace2d_indices!(row_indices, global_row, g)
    row_values[1] = 4.0 - (5-row_n_elems)
    Tpetra.replaceGlobalValues(A, global_row, Teuchos.ArrayView(row_indices,row_n_elems), Teuchos.ArrayView(row_values,row_n_elems))
  end
end
```

### Timing comparison for 1000 x 1000 matrix (in ms)
|                  |C++  |Julia|
|------------------|-----|-----|
|Graph construction|190.7|115.2|
|Source term       |41.17|32.01|
|Matrix filling    |137.1|102.5|
|Dirichlet setup   |0.899|0.761|
|Check time        |41.06|29.42|